In [ ]:
## Breast Cancer LASSO Exploration
## Prepare workspace
from scipy.io import loadmat
import numpy as np
import math
X = loadmat("BreastCancer.mat")['X']
y = loadmat("BreastCancer.mat")['y']

In [ ]:
def ista_solve_hot( A, d, la_array ):
    # ista_solve_hot: Iterative soft-thresholding for multiple values of
    # lambda with hot start for each case - the converged value for the previous
    # value of lambda is used as an initial condition for the current lambda.
    # this function solves the minimization problem
    # Minimize |Ax-d|_2^2 + lambda*|x|_1 (Lasso regression)
    # using iterative soft-thresholding.
    max_iter = 10**4
    tol = 10**(-3)
    tau = 1/np.linalg.norm(A,2)**2
    n = A.shape[1]
    w = np.zeros((n,1))
    num_lam = len(la_array)
    X = np.zeros((n, num_lam))
    for i, each_lambda in enumerate(la_array):
        for j in range(max_iter):
            z = w - tau*(A.T@(A@w-d))
            w_old = w
            w = np.sign(z) * np.clip(np.abs(z)-tau*each_lambda/2, 0, np.inf)
            X[:, i:i+1] = w
            if np.linalg.norm(w - w_old) < tol:
                break
    return X

def ridge(A, d, la_array):
    dim = A.shape[0]
    n = A.shape[1]
    w = np.zeros((n,1))
    num_lam = len(la_array)
    X = np.zeros((n, num_lam))
    for i, each_lambda in enumerate(la_array):
        w = A.transpose()@np.linalg.inv(
            A@A.transpose() + each_lambda*np.identity(dim))@d
        X[:, i:i+1] = w
    return X

def error_rate(X, y, weights):
    err = 0
    y_hat = np.sign(X@weights)
    for i in range(len(y_hat)):
      if (y[i] != y_hat[i]):
        err+=1
    return err/len(y_hat)

In [ ]:
##  10-fold CV 

# each row of setindices denotes the starting an ending index for one
# partition of the data: 5 sets of 30 samples and 5 sets of 29 samples
setindices = [[1,30],[31,60],[61,90],[91,120],[121,150],
              [151,179],[180,208],[209,237],[238,266],[267,295]]

# each row of holdoutindices denotes the partitions that are held out from
# the training set
holdoutindices = [[1,2],[2,3],[3,4],[4,5],[5,6],[7,8],[9,10],[10,1]]

cases = len(holdoutindices)

# be sure to initiate the quantities you want to measure before looping
# through the various training, validation, and test partitions
lambda_list = np.logspace(-6, math.log10(20))

misclass_L = np.zeros(cases)
misclass_2 = np.zeros(cases)
squared_error_L = np.zeros(cases)
squared_error_2 = np.zeros(cases)

# Loop over various cases
for j in range(cases):
    # row indices of first validation set
    v1_ind = np.arange(setindices[holdoutindices[j][0]-1][0]-1,
                       setindices[holdoutindices[j][0]-1][1])
    
    # row indices of second validation set
    v2_ind = np.arange(setindices[holdoutindices[j][1]-1][0]-1,
                       setindices[holdoutindices[j][1]-1][1])
    
    # row indices of training set
    trn_ind = list(set(range(295))-set(v1_ind)-set(v2_ind))
    
    # define matrix of features and labels corresponding to first
    # validation set
    Av1 = X[v1_ind,:]
    bv1 = y[v1_ind]
    
    # define matrix of features and labels corresponding to second
    # validation set
    Av2 = X[v2_ind,:]
    bv2 = y[v2_ind]
    
    # define matrix of features and labels corresponding to the 
    # training set
    At = X[trn_ind,:]
    bt = y[trn_ind]
    
    print(len(v1_ind), len(v2_ind), len(trn_ind))

    # Use training data to learn classifier
    W_L = ista_solve_hot(At,bt,lambda_list) #LASSO
    W_2 = ridge(At, bt, lambda_list) #ridge regression

    # Find best lambda value using first validation set
    error_L = [error_rate(Av1, bv1, W_L[:,i]) for i in range(50)]
    error_2 = [error_rate(Av1, bv1, W_2[:,i]) for i in range(50)]
    # index of best weights
    best_L = np.argsort(error_L)[0]
    best_2 = np.argsort(error_2)[0]

    # performance on second validation set
    best_error_L = error_rate(Av2, bv2, W_L[:,best_L])
    best_error_2 = error_rate(Av2, bv2, W_2[:,best_2])
    misclass_L[j] = best_error_L
    misclass_2[j] = best_error_2

    squared_L = np.linalg.norm(
        ((Av2@W_L[:,best_L]).reshape(len(bv2),1) - bv2),ord=2)**2
    squared_2 = np.linalg.norm(
        ((Av2@W_2[:,best_2]).reshape(len(bv2),1) - bv2),ord=2)**2
    squared_error_L[j] = squared_L
    squared_error_2[j] = squared_2


30 30 235
30 30 235
30 30 235
30 30 235
30 29 236
29 29 237
29 29 237
29 30 236


In [ ]:
print("For LASSO:")
print("Average misclassification error:", np.average(misclass_L))
print("Average squared error:", np.average(squared_error_L))
print("\n")
print("For ridge regression:")
print("Average misclassification error:", np.average(misclass_2))
print("Average squared error:", np.average(squared_error_2))

For LASSO:
Average misclassification error: 0.2997126436781609
Average squared error: 23.440022647942314


For ridge regression:
Average misclassification error: 0.3038793103448276
Average squared error: 26.52090910565953
